# SNPE & RF

running this notebook will requre the following repositories to be locally installed and on the specified branches:
- delfi 
- lfimodels / maprf_elife 
- maprf_mn  / elife      


In [ ]:
#%%capture
%matplotlib inline

import delfi.distribution as dd
from delfi.utils.viz import plot_pdf, plot_marg_axes
from lfimodels.maprf.utils import setup_sim, get_data_o

import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches

%run -i common.ipynb

#import matplotlib.pyplot as plt
#import numpy as np

root_path     = PATH_DROPBOX + 'materials/fig2/mapRF'
root_path_glm = PATH_DROPBOX + 'materials/fig2/GLM' 

In [ ]:
# FIGURE AND GRID
FIG_HEIGHT_MM = 65
FIG_WIDTH_MM = 1.15*FIG_WIDTH_MM  # set in common notebook to a default value for all figures
FIG_N_ROWS = 1
ROW_1_NCOLS = 2
ROW_1_HEIGHT_MM = FIG_HEIGHT_MM / FIG_N_ROWS
ROW_1_WIDTH_COL_1_MM = 1.1*FIG_WIDTH_MM / ROW_1_NCOLS
ROW_1_WIDTH_COL_2_MM = 0.9*FIG_WIDTH_MM / ROW_1_NCOLS
# ROW_2_NCOLS = 1
# ROW_2_HEIGHT_MM = FIG_HEIGHT_MM / FIG_N_ROWS
# ROW_2_WIDTH_COL_1_MM = FIG_WIDTH_MM / ROW_2_NCOLS

PATH_DROPBOX_FIGS = PATH_DROPBOX + 'figs/'

# PATHS
PANEL_A = PATH_DROPBOX_FIGS + 'fig2_a.svg'

fontsize=14


In [ ]:
svg(PANEL_A)

# temporal GLM results

In [ ]:
tmp = np.load(root_path_glm + '/results/SNPE_B/glm_5k_elife_prior_gp_run_1_round5_param10_SNPE_posterior.npy')[()]
posterior, proposal, prior = tmp['posterior'], tmp['proposal'], tmp['prior']

tmp = np.load(root_path_glm+'/results/SNPE_B/ground_truth_data.npy')[()]
obs_stats, pars_true, labels_params = tmp['obs_stats'],  tmp['pars_true'], tmp['labels_params']

samples = np.load(root_path_glm+'/results/MCMC/sam.npz')['arr_0'].T

plot_post = posterior
plot_prior = prior
pars_raw   = pars_true
labels_params = params_ls = ['bias', 'f1', 'f2', 'f3', 'f4', 'f5','f6','f7','f8','f9']

lims_samples = np.array([[-3, -3, -3, -3, -3, -3, -3, -3, -3, -3], 
                         [ 3,  3,  3,  3,  3,  3,  3,  3,  3,  3]]).T

lims_post    = np.array([[-3, -3, -3, -3, -3, -3, -3, -3, -3, -3], 
                         [ 3,  3,  3,  3,  3,  3,  3,  3,  3,  3]]).T


obs_stats, pars_true, params_ls

## panel for summary statistics

In [ ]:
plt.figure(figsize=(2, 2))
plt.plot(np.arange(obs_stats.size), obs_stats.flatten(), 'bo-')
plt.title('cross-correlation', fontsize=fontsize)
#plt.xlabel('#summary', fontsize=fontsize)
#plt.ylabel( 'value', fontsize=fontsize)
plt.xticks([0,  9], fontsize=fontsize)
plt.yticks([0, 10], fontsize=fontsize)
plt.text(4, -11, r'$\Delta{}t$', fontsize=fontsize)
plt.tight_layout()
plt.box('off')

PANEL_B_1 = PATH_DROPBOX_FIGS + 'fig2_b_1.svg'
plt.savefig(PANEL_B_1, facecolor=plt.gcf().get_facecolor(), transparent=True)


## panel for parameters

In [ ]:
plt.figure(figsize=(2, 2))
plt.plot(np.arange(pars_true.size)+1, pars_true.flatten(), 'ro-')
plt.title('ground-truth', fontsize=fontsize)
#plt.ylabel( 'value', fontsize=fontsize)
#plt.xlabel( '#param', fontsize=fontsize)
plt.xticks([1, 6, 10], np.array(params_ls)[np.array([0,5,9])], fontsize=fontsize)
plt.yticks([-2, 0, 2], fontsize=fontsize)
plt.tight_layout()
plt.box('off')

PANEL_B_2 = PATH_DROPBOX_FIGS + 'fig2_b_2.svg'
plt.savefig(PANEL_B_2, facecolor=plt.gcf().get_facecolor(), transparent=True)


## version with all marginals + selected pair-wise marginals

In [ ]:
plt.figure(figsize=(7.5,4))

styles = mpatches.ArrowStyle.get_styles()

dx, MM, nxh, nxp = 10, len(lims_samples), 70, 200
M, M_ = MM, MM * 1.15

idx = np.array([0,1,2,3,4,5,6,7,8,9])
pairs = [[0,1], # bias  vs first filter
         [1,2], # 
         [2,4], # 
         [4,8]] # 

gs = gridspec.GridSpec(2, 3, width_ratios=[2, 1, 1], height_ratios=[1,1])
for j in range(4):

    ax = plt.subplot(gs[(j%2),1+(j>1)])
    plot_marg_axes(ax=ax, 
                   i=pairs[j][0], j=pairs[j][1],
                   pdf=plot_post, 
                   samples=samples, 
                   lims=lims_samples, 
                   gt=pars_raw, 
                   bins=100, 
                   resolution=100,
                   cmap=None,
                   contours=True,
                   contour_levels=(0.68, 0.95),
                   contour_colors=('w','y'),
                   scatter=False,
                   scatter_color='gray',
                   scatter_alpha=0.2)
    plt.xlabel(labels_params[pairs[j][1]], fontsize=fontsize)
    plt.ylabel(labels_params[pairs[j][0]], fontsize=fontsize)
plt.tight_layout()    


stylename, styleclass = sorted(styles.items())[0]            
gs = gridspec.GridSpec(1, 3, width_ratios=[2.2, 1, 1], height_ratios=[1])
ax = plt.subplot(gs[0,0])

xh = np.linspace(0, 1, nxh) 
for i in idx:
    
    S = samples[:,idx[i]].copy()
    mm, MM = lims_post[idx[i]] #S.min(), S.max()
    yy = np.linspace(mm, MM, nxp)
    S = (S - mm) / (MM - mm)
    xx = np.linspace(0, 1, nxp) 
    
    h, xh_ = np.histogram(S, bins=xh, normed=True)
    Mh = h.max() / (0.9*dx)
    h = h / Mh
    
    """
    ax.barh(bottom=xh[:-1], width=h, left=(M-i)*dx, color='orange', height=1.1*np.diff(xh)[0])    
    ff = plot_post.eval(yy.reshape(-1,1), ii=[idx[i]], log=False)*(MM - mm)
    ax.plot(ff/Mh + (M-i)*dx, xx, color=COL['SNPE'], linewidth=2.5)
    ff = plot_prior.eval(yy.reshape(-1,1), ii=[idx[i]], log=False)*(MM - mm)
    ax.plot(ff/Mh + (M-i)*dx, xx, color=COL['IBEA'], linewidth=1.5)
    ax.text(dx*(M-i), -0.05, params_ls[idx[i]], fontsize=fontsize, rotation=45)    
    ax.plot(dx*(M-i), (pars_raw[idx[i]]-mm)/(MM-mm), 'r*', markersize=9)
    """
    
    ax.bar(left=xh[:-1], height=h, bottom=(M-i)*dx, color='orange', width=np.diff(xh)[0], orientation='vertical')
    ff = plot_post.eval(yy.reshape(-1,1), ii=[idx[i]], log=False)*(MM - mm)
    ax.plot(xx, ff/Mh + (M-i)*dx, color=COL['SNPE'], linewidth=2.5)
    ff = plot_prior.eval(yy.reshape(-1,1), ii=[idx[i]], log=False)*(MM - mm)
    ax.plot(xx, ff/Mh + (M-i)*dx, color=COL['IBEA'], linewidth=1.5)
    ax.text(-0.15, dx*(M-i+0.35) , params_ls[idx[i]], fontsize=fontsize, rotation=20)    
    ax.plot((pars_raw[idx[i]]-mm)/(MM-mm), dx*(M-i), 'r*', markersize=9)    
    
ax.axis('off')    


"""
# in case variables are grouped by transformation ( idx = [1,0,6,3,5,4,2,7,8] )
plt.plot([0.9*dx, 4.5*dx], [-0.1, -0.1], 'k')
plt.text(2.4*dx, -0.17, 'logit-normal', fontsize=fontsize)
plt.plot([4.9*dx, 7.5*dx], [-0.12, -0.12], 'k')
plt.text(5.9*dx, -0.17, 'log-normal', fontsize=fontsize)
plt.plot([7.9*dx, 9.5*dx], [-0.11, -0.11], 'k')
plt.text(8.4*dx, -0.17, 'normal', fontsize=fontsize)
"""

PANEL_B_3 = PATH_DROPBOX_FIGS + 'fig2_b_3.svg'
plt.savefig(PANEL_B_3, facecolor=plt.gcf().get_facecolor(), transparent=True)

plt.show()


## panel for posterior samples 

In [ ]:

n_draws=10 
plt.figure(figsize=(4,4))
for i in range(n_draws):
    
    plt.plot(np.arange(pars_true.size)+1, posterior.gen(1).flatten(), 'o-', color=COL['SNPE'], alpha=0.5)

plt.xticks([1, 4, 7, 10], np.array(params_ls)[np.array([0,3,6,9])], fontsize=fontsize)
plt.yticks([-2, 0, 2], fontsize=fontsize)
plt.tight_layout()
plt.box('off')    
plt.title('posterior samples', fontsize=fontsize)

plt.plot(np.arange(pars_true.size)+1, pars_true.flatten(), 'ro-', linewidth=2)


plt.tight_layout()

PANEL_B_4 = PATH_DROPBOX_FIGS + 'fig2_b_4.svg'
plt.savefig(PANEL_B_4, facecolor=plt.gcf().get_facecolor(), transparent=True)
plt.show()


# mapRF results

In [ ]:
## training data and true parameters, data, statistics
seed = 42
idx_cell = 6 # load toy cell number i 

tmp = np.load(root_path+'/results/SNPE/toycell_6/ground_truth_data.npy')[()]
obs_stats, pars_true, rf = tmp['obs_stats'],  tmp['pars_true'], tmp['rf']

sim_info = np.load(root_path +'/results/sim_info.npy')[()]
d, params_ls = sim_info['d'], sim_info['params_ls']

assert obs_stats[0,-1] == 307 # the cell we want to work with should have this number of spikes

labels_params = ['bias', 'gain', 'phase', 'freq', 'angle', 'ratio', 'width', 'xo', 'yo']


# load SNPE results

In [ ]:
tmp = np.load(root_path + '/results/SNPE/toycell_6/maprf_100k_elife_prior01_run_9_round4_param9_nosvi_CDELFI_posterior.npy')[()]
posterior, proposal, prior = tmp['posterior'], tmp['proposal'], tmp['prior']

plot_prior = dd.TransformedNormal(m=prior.m, S = prior.S,
                            flags=[0,0,2,1,2,1,1,2,2],
                            lower=[0,0,0,0,0,0,0,-1,-1], upper=[0,0,np.pi,0,2*np.pi,0,0,1,1]) 

plot_post = dd.mixture.TransformedGaussianMixture.MoTG(
                            ms= [posterior.xs[i].m for i in range(posterior.n_components)], 
                            Ss =[posterior.xs[i].S for i in range(posterior.n_components)],
                            a = posterior.a, 
                            flags=[0,0,2,1,2,1,1,2,2],
                            lower=[0,0,0,0,0,0,0,-1,-1], upper=[0,0,np.pi,0,2*np.pi,0,0,1,1]) 

lims_post = np.array([[-1.5, -1.1, .001,         0,          .001, 0, 0, -.999, -.999], 
                 [ 1.5,  1.1, .999*np.pi, 2.5,   1.999*np.pi, 2, 4., .999,   .999]]).T

# load comparison against MCMC sampler

In [ ]:
n_samples=1000000
path = root_path + '/results/MCMC/'
savefile = path + 'toycell_' + str(idx_cell) + '/maprf_MCMC_prior01_run_1_'+ str(n_samples)+'samples_param9_5min.npy'
tmp = np.load(savefile)[()]

T, params_dict_true = tmp['T'], tmp['params_dict_true']

params_ls = ['bias', 'gain', 'phase', 'freq','angle','ratio','width', 'xo', 'yo']
samples = np.hstack([np.atleast_2d(T[key].T).T for key in params_ls])

def symmetrize_sample_modes(samples):

    assert samples.ndim==2 and samples.shape[1] == 9 

    # assumes phase in [0, pi]
    assert np.min(samples[:,2]) >= 0. and np.max(samples[:,2] <= np.pi)
    # assumes angle in [0, 2*pi]
    assert np.min(samples[:,4]) >= 0. and np.max(samples[:,4] <= 2*np.pi)
    # assumes freq, ratio and width > 0
    assert np.all(np.min(samples[:,np.array([3,5,6])], axis=0) >= 0.)

    samples1 = samples.copy()
    idx = np.where( samples[:,4] > np.pi )[0]
    samples1[idx,4] = samples1[idx,4] - np.pi
    idx = np.where( samples[:,4] < np.pi )[0]
    samples1[idx,4] = samples1[idx,4] + np.pi
    #samples1[:,2] = np.pi - samples1[:,2]
    samples_all = np.vstack((samples, samples1))[::2, :]

    samples1 = samples_all.copy()
    samples1[:,1] = - samples1[:,1] 
    samples1[:,2] = np.pi - samples1[:,2] 
    samples_all = np.vstack((samples_all, samples1))[::2, :]

    return samples_all

samples = symmetrize_sample_modes(samples)

pars_raw = np.array([ params_dict_true['glm']['bias'],
                      params_dict_true['kernel']['s']['gain'],
                      params_dict_true['kernel']['s']['phase'] + 0.05, # remove phase a bit from left interval border
                      params_dict_true['kernel']['s']['angle'],
                      params_dict_true['kernel']['s']['freq'],
                      params_dict_true['kernel']['s']['ratio'],
                      params_dict_true['kernel']['s']['width'],
                      params_dict_true['kernel']['l']['xo'],
                      params_dict_true['kernel']['l']['yo'] ])

lims_samples = np.array([[-.5, -1.1, .00001*np.pi, 0, 0.301*np.pi, 0, 0,-0.1,-0.0], 
                         [ .1,  1.1, .99999*np.pi, 3, 1.699*np.pi, 3, 5, 0.4, 0.5]]).T

## panel for summary statistics

In [ ]:
plt.figure(figsize=(2, 2))
plt.imshow(obs_stats[0,:-1].reshape(d,d), interpolation='None', cmap='gray')
plt.title('STA', fontsize=fontsize)
plt.tight_layout()
plt.axis('off')

# option to add contours of ground-truth RF
add_gt = False
if add_gt:    
    rfm = g.model.params_to_rf(pars_true.reshape(-1))[0]
    plt.contour(rfm, levels=[lvls[0]*rfm.min(), lvls[1]*rfm.max()], colors='r')

PANEL_C_1 = PATH_DROPBOX_FIGS + 'fig2_c_1.svg'
plt.savefig(PANEL_C_1, facecolor=plt.gcf().get_facecolor(), transparent=True)


## panel for parameters

In [ ]:
plt.figure(figsize=(2, 2))
plt.imshow(rf, interpolation='None')
plt.title('ground-truth', fontsize=fontsize)
plt.tight_layout()
plt.axis('off')

PANEL_C_2 = PATH_DROPBOX_FIGS + 'fig2_c_2.svg'
plt.savefig(PANEL_C_2, facecolor=plt.gcf().get_facecolor(), transparent=True)
plt.show()


## panel for (partial) posterior

In [ ]:
plt.figure(figsize=(7.5,4))

styles = mpatches.ArrowStyle.get_styles()

dx, MM, nxh, nxp = 10, len(lims_samples), 70, 200
M, M_ = MM, MM * 1.15

idx = np.array([0,1,6,3,5,2,4,8,7])
pairs = [[0,1], # bias  vs gain
         [2,1], # gain  vs phase
         [2,4], # phase vs angle
         [8,7]] # xo vs yo
         #[5,6]] # ratio vs width


gs = gridspec.GridSpec(2, 3, width_ratios=[2, 1, 1], height_ratios=[1,1])
for j in range(4):

    ax = plt.subplot(gs[(j%2),1+(j>1)])
    plot_marg_axes(ax=ax, 
                   i=pairs[j][0], j=pairs[j][1],
                   pdf=plot_post, 
                   samples=samples, 
                   lims=lims_samples, 
                   gt=pars_raw, 
                   bins=100, 
                   resolution=100,
                   cmap=None,
                   contours=True,
                   contour_levels=(0.68, 0.95),
                   contour_colors=('w','y'),
                   scatter=False,
                   scatter_color='gray',
                   scatter_alpha=0.2)
    plt.xlabel(labels_params[pairs[j][1]], fontsize=fontsize)
    plt.ylabel(labels_params[pairs[j][0]], fontsize=fontsize)
plt.tight_layout()    



stylename, styleclass = sorted(styles.items())[0]            
gs = gridspec.GridSpec(1, 3, width_ratios=[2.2, 1, 1], height_ratios=[1])
ax = plt.subplot(gs[0,0])

xh = np.linspace(0, 1, nxh) 
for i in idx:
    
    S = samples[:,idx[i]].copy()
    mm, MM = lims_post[idx[i]] #S.min(), S.max()
    yy = np.linspace(mm, MM, nxp)
    S = (S - mm) / (MM - mm)
    xx = np.linspace(0, 1, nxp) 

    h, xh_ = np.histogram(S, bins=xh, normed=True)
    Mh = h.max() / (0.9*dx)
    h = h / Mh
    
    """
    #ax.plot([(M-i)*dx, (M-i)*dx], [0, 1], 'k')    
    ax.barh(bottom=xh[:-1], width=h, left=(M-i)*dx, color='orange', height=1.1*np.diff(xh)[0])    
    ff = plot_post.eval(yy.reshape(-1,1), ii=[idx[i]], log=False)*(MM - mm)
    ax.plot(ff/Mh + (M-i)*dx, xx, color=COL['SNPE'], linewidth=2.5)
    ff = plot_prior.eval(yy.reshape(-1,1), ii=[idx[i]], log=False)*(MM - mm)
    ax.plot(ff/Mh + (M-i)*dx, xx, color=COL['IBEA'], linewidth=1.5)
    ax.text(dx*(M-i), -0.05, params_ls[idx[i]], fontsize=fontsize, rotation=45)    
    ax.plot(dx*(M-i), (pars_raw[idx[i]]-mm)/(MM-mm), 'r*', markersize=9)
    """
    
    #ax.plot([0, 1], [(M-i)*dx, (M-i)*dx], 'k')    
    ax.bar(left=xh[:-1], height=h, bottom=(M-i)*dx, color='orange', width=np.diff(xh)[0], orientation='vertical')
    ff = plot_post.eval(yy.reshape(-1,1), ii=[idx[i]], log=False)*(MM - mm)
    ax.plot(xx, ff/Mh + (M-i)*dx, color=COL['SNPE'], linewidth=2.5)
    ff = plot_prior.eval(yy.reshape(-1,1), ii=[idx[i]], log=False)*(MM - mm)
    ax.plot(xx, ff/Mh + (M-i)*dx, color=COL['IBEA'], linewidth=1.5)
    ax.text(-0.22, dx*(M-i+0.35) , params_ls[idx[i]], fontsize=fontsize, rotation=20)    
    ax.plot((pars_raw[idx[i]]-mm)/(MM-mm), dx*(M-i), 'r*', markersize=9)    
    
ax.axis('off')    

"""
# in case variables are grouped by transformation ( idx = [1,0,6,3,5,4,2,7,8] )
plt.plot([0.9*dx, 4.5*dx], [-0.1, -0.1], 'k')
plt.text(2.4*dx, -0.17, 'logit-normal', fontsize=fontsize)
plt.plot([4.9*dx, 7.5*dx], [-0.12, -0.12], 'k')
plt.text(5.9*dx, -0.17, 'log-normal', fontsize=fontsize)
plt.plot([7.9*dx, 9.5*dx], [-0.11, -0.11], 'k')
plt.text(8.4*dx, -0.17, 'normal', fontsize=fontsize)
"""

PANEL_C_3 = PATH_DROPBOX_FIGS + 'fig2_c_3.svg'
plt.savefig(PANEL_C_3, facecolor=plt.gcf().get_facecolor(), transparent=True)

plt.show()


## panel for posterior samples 

In [ ]:

# this snippet of code requires the mapRF repository (to instantiate g.model)
g, prior, d = setup_sim(seed, path=root_path)
filename = root_path + '/results/toy_cells/toy_cell_' + str(idx_cell) + '.npy'
obs_stats, pars_true = get_data_o(filename, g, seed)
rf = g.model.params_to_rf(pars_true)[0]

lvls, n_draws=[0.5, 0.5], 10 
plt.figure(figsize=(4,4))
plt.imshow(obs_stats[0,:-1].reshape(d,d), interpolation='None', cmap='gray')
for i in range(n_draws):
    rfm = g.model.params_to_rf(posterior.gen().reshape(-1))[0]
    plt.contour(rfm, levels=[lvls[0]*rfm.min(), lvls[1]*rfm.max()], colors=[COL['SNPE']], linewidth=2)
    plt.hold(True)
plt.title('posterior samples', fontsize=fontsize)

rfm = g.model.params_to_rf(pars_true.reshape(-1))[0]
plt.contour(rfm, levels=[lvls[0]*rfm.min(), lvls[1]*rfm.max()], colors='r', linewidth=2.5)
plt.tight_layout()
plt.axis('off')

PANEL_C_4 = PATH_DROPBOX_FIGS + 'fig2_c_4.svg'
plt.savefig(PANEL_C_4, facecolor=plt.gcf().get_facecolor(), transparent=True)
plt.show()


# compose figure

In [ ]:
# FIGURE and GRID
FIG_HEIGHT_MM = 200
FIG_WIDTH_MM = FIG_WIDTH_MM  # set in NIPS2017 notebook to a default value for all figures
FIG_N_ROWS = 4
ROW_1_NCOLS = 1
ROW_1_HEIGHT_MM =     0.6 * (FIG_HEIGHT_MM / FIG_N_ROWS )
ROW_1_WIDTH_COL_1_MM =  3 *(FIG_WIDTH_MM / ROW_1_NCOLS)

ROW_2_NCOLS = 3
ROW_2_HEIGHT_MM = FIG_HEIGHT_MM / FIG_N_ROWS
ROW_2_WIDTH_COL_1_MM = 0.3*FIG_WIDTH_MM / ROW_2_NCOLS
ROW_2_WIDTH_COL_2_MM = 1.6*FIG_WIDTH_MM / ROW_2_NCOLS
ROW_2_WIDTH_COL_3_MM = 1.1 *FIG_WIDTH_MM / ROW_2_NCOLS
ROW_3_NCOLS = 1
ROW_3_HEIGHT_MM = FIG_HEIGHT_MM / FIG_N_ROWS
ROW_3_WIDTH_COL_1_MM = FIG_WIDTH_MM / ROW_3_NCOLS
ROW_3_WIDTH_COL_2_MM = FIG_WIDTH_MM / ROW_3_NCOLS
ROW_3_WIDTH_COL_3_MM = FIG_WIDTH_MM / ROW_3_NCOLS

In [ ]:
fig = create_fig(FIG_WIDTH_MM, FIG_HEIGHT_MM)

yoffset = -2.2
xoffset = -1.
fig = add_svg(fig, PANEL_A, 
              5 + xoffset, 
              0 + yoffset)


yoffset = 30
fig = add_svg(fig, PANEL_B_2, 
              0 + xoffset, 
              4 + yoffset)
fig = add_svg(fig, PANEL_B_1, 
              0 + xoffset , 
              ROW_1_HEIGHT_MM + yoffset) 
fig = add_svg(fig, PANEL_B_3, 
              ROW_2_WIDTH_COL_1_MM + xoffset + 10,                        
              yoffset + 2.1)  
fig = add_svg(fig, PANEL_B_4, 
              ROW_2_WIDTH_COL_1_MM + ROW_2_WIDTH_COL_2_MM + xoffset, 
              yoffset + 1.8)

yoffset = 2.5
fig = add_label(fig, 'A', 
                0, 
                0 + yoffset)
yoffset = 5
fig = add_label(fig, 'B', 
                0, 
                ROW_1_HEIGHT_MM + yoffset)
fig = add_label(fig, 'C', 
                ROW_2_WIDTH_COL_1_MM + 5, 
                ROW_1_HEIGHT_MM + yoffset)
fig = add_label(fig, 'D', 
                ROW_2_WIDTH_COL_1_MM + ROW_2_WIDTH_COL_2_MM, 
                ROW_1_HEIGHT_MM + yoffset)

#yoffset = 30
fig = add_svg(fig, PANEL_C_2, 
              0 + xoffset, 
              0 + yoffset + ROW_1_HEIGHT_MM + ROW_2_HEIGHT_MM + 2)
fig = add_svg(fig, PANEL_C_1, 
              0 + xoffset, 
              ROW_1_HEIGHT_MM + ROW_1_HEIGHT_MM + ROW_2_HEIGHT_MM + yoffset - 2.5) 
fig = add_svg(fig, PANEL_C_3, 
              ROW_2_WIDTH_COL_1_MM + xoffset + 10,                   
              yoffset + ROW_2_HEIGHT_MM+ ROW_1_HEIGHT_MM + 2.1)  
fig = add_svg(fig, PANEL_C_4, 
              ROW_2_WIDTH_COL_1_MM + ROW_2_WIDTH_COL_2_MM + xoffset, 
              yoffset + ROW_2_HEIGHT_MM + ROW_1_HEIGHT_MM + 1.8)

yoffset = 11
fig = add_label(fig, 'E', 
                0,                        
                ROW_1_HEIGHT_MM + ROW_2_HEIGHT_MM + yoffset)
fig = add_label(fig, 'F', 
                ROW_2_WIDTH_COL_1_MM + 5, 
                ROW_1_HEIGHT_MM + ROW_2_HEIGHT_MM + yoffset)
fig = add_label(fig, 'G', 
                ROW_2_WIDTH_COL_1_MM + ROW_2_WIDTH_COL_2_MM, 
                ROW_1_HEIGHT_MM + ROW_2_HEIGHT_MM + yoffset)



yoffset = 21
fig = add_label(fig, 'H', 
                0,                        
                ROW_1_HEIGHT_MM + ROW_2_HEIGHT_MM + ROW_3_HEIGHT_MM + yoffset)
fig = add_label(fig, 'I', 
                ROW_2_WIDTH_COL_1_MM + 5, 
                ROW_1_HEIGHT_MM + ROW_2_HEIGHT_MM + ROW_3_HEIGHT_MM + yoffset)
fig = add_label(fig, 'J', 
                ROW_2_WIDTH_COL_1_MM + ROW_2_WIDTH_COL_2_MM, 
                ROW_1_HEIGHT_MM + ROW_2_HEIGHT_MM + ROW_3_HEIGHT_MM + yoffset)


if False:
    fig = add_grid(fig, 2, 2)
    fig = add_grid(fig, 160/3, 10, font_size_px=0.0001)

PATH_SVG = PATH_DROPBOX_FIGS + 'fig2.svg'
fig.save(PATH_SVG)
svg(PATH_SVG)
!$INKSCAPE --export-pdf $PATH_DROPBOX_FIGS/fig2.pdf $PATH_SVG

# supplementary figure: full posterior

In [ ]:
fig, _ = plot_pdf(plot_post, pdf2=plot_prior, lims=lims_post, gt=plot_post._f(pars_true.reshape(1,-1)).reshape(-1), 
                  figsize=(16,16), resolution=100,
                  contour_levels=[0.68, 0.95],
                  contour_colors=('w', 'y'),
                  hist_color='orange',
                  pdf1_color=COL['SNPE'],                  
                  pdf2_color=COL['GT'],                  
                  labels_params=labels_params)


In [ ]:
fig, _ = plot_pdf(plot_post,  pdf2=plot_prior, lims=lims_samples, gt=pars_raw.reshape(-1), figsize=(16,16), 
                  resolution=100, samples=samples,
                  contour_levels=[0.68, 0.95],
                  contour_colors=('w', 'y'),
                  hist_color='orange',
                  pdf1_color=COL['SNPE'],                  
                  pdf2_color=COL['GT'],                        
                  labels_params=labels_params);
fig.savefig('posterior_final_full.pdf')